# tesnorQTL analysis for all chromosomes

## Import packages

In [ ]:
import pandas as pd
import torch
import tensorqtl
from tensorqtl import genotypeio, cis, trans, post, pgen
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"torch: {torch.__version__} (CUDA {torch.version.cuda}), device: {device}")
print(f"pandas {pd.__version__}")
import os.path

## Run the analysis for all chromosomes

In [ ]:
list_of_df_cis = []
full_table_cis = pd.DataFrame()

list_of_df_trans = []
full_table_trans = pd.DataFrame()

for x in range(1,23):
    
    # Set paths
    plink_prefix_path = f'/vcfs/brca_chr{x}'
    methylation_bed = 'methylation_residuals_3.bed.gz'
    prefix ='/meQTL_results/meQTL'
    
    # Load phenotypes, covariates 
    phenotype_df, phenotype_pos_df = tensorqtl.read_phenotype_bed(methylation_bed)
    # Covariates file not needed since we adjusted for covariates when obtaining methylation residuals with PEER
    
    pr = genotypeio.PlinkReader(f'/vcfs/brca_chr{x}')
    # Load genotypes and variants into data frames
    genotype_df = pr.load_genotypes()
    variant_df = pr.bim.set_index('snp')[['chrom', 'pos']]
    
    
    ####### cis-meQTL: nominal p-values for all variant-phenotype pairs per chrosomome
    cis.map_nominal(genotype_df, variant_df,
                phenotype_df.loc[phenotype_pos_df['chr'] == f'chr{x}'],
                phenotype_pos_df.loc[phenotype_pos_df['chr'] == f'chr{x}'],
                prefix, window=1000000)  
    # load results
    pairs_df = pd.read_parquet(f'{prefix}.cis_qtl_pairs.chr{x}.parquet')
    pairs_df
    
    # Resave only significant cis-meQTLs per chromosome as csv file
    pairs_df[pairs_df['pval_nominal'] <= 2e-5].to_csv(f'{prefix}.cis_qtl_pairs_chr{x}.csv')
    
    # Create list with significant cis-meQTLs across all chrosomes
    list_of_df_cis.append(pairs_df[(pairs_df['pval_nominal'] < 2e-5)]) 
    
    ####### trans-meQTLs #############
    trans_df = trans.map_trans(genotype_df, phenotype_df,
                           return_sparse=True, pval_threshold=2e-5, maf_threshold=0,
                            batch_size=20000) 
                                              
 
    # remove cis-associations
    trans_df = trans.filter_cis(trans_df, phenotype_pos_df.T.to_dict(), variant_df, window=1000000) 
    
    # save trans meQTLs
    trans_df.to_csv(f'{prefix}.trans_chr{x}.csv')
    
    # Create list with significant trans-meQTLs across all chrosomes
    list_of_df_trans.append(trans_df) 

In [6]:
# Save overall results
full_df_cis = pd.concat(list_of_df_cis,ignore_index=True)
full_df_trans = pd.concat(list_of_df_trans,ignore_index=True)

full_df_cis.to_csv(f'{prefix}_cis_sig_all_3.csv')
full_df_trans.to_csv(f'{prefix}_trans_sig_all_3.csv')